In [3]:
!pip install statsmodels


Defaulting to user installation because normal site-packages is not writeable


In [5]:
import pandas as pd
import numpy as np

# 统计建模 & 机器学习
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# =========================
# 1. 读取数据
# =========================

df = pd.read_csv("ng_with_EXP.csv")

y_col = "Exp_flag"

# 结构变量
struct_vars = [
    "BtE400",
    "Cent_norm"
]

# 服务变量
service_vars = [
    "400tram_ratio",
    "400bus_ratio",
    "metroPointCount",
    "busPoint_Count",
    "access_gap"
]

# 人口变量
pop_vars = [
    "IMD_rank",
    "olderratio",
    "popdense"
]

X_cols = struct_vars + service_vars + pop_vars

# 只保留需要的列
data = df[X_cols + [y_col]].copy()
data = data.replace([np.inf, -np.inf], np.nan).dropna()

X = data[X_cols]
y = data[y_col].astype(int)

print("有效样本数:", len(data))

# =========================
# 2. 标准化
# =========================

scaler = StandardScaler()
X_scaled_array = scaler.fit_transform(X)

X_scaled = pd.DataFrame(
    X_scaled_array,
    columns=X_cols,
    index=X.index
)

# =========================
# 3. Logistic 回归
# =========================

X_logit = sm.add_constant(X_scaled)

# 增大迭代次数，使用 lbfgs
# 带 L1 惩罚的逻辑回归，专门对付 quasi-separation
logit_pen = sm.Logit(y, X_logit)

logit_res = logit_pen.fit_regularized(
    method="l1",     # L1 正则
    maxiter=500
)

print(logit_res.summary())

# 计算 OR
coef = logit_res.params
OR = np.exp(coef)
print("\nOdds Ratios (penalized):")
print(OR)

# 预测概率
data["pred_prob"] = logit_res.predict(X_logit)


# 计算 OR
params = logit_res.params
conf = logit_res.conf_int()
conf.columns = ["lower", "upper"]

OR_table = pd.DataFrame({
    "variable": params.index,
    "OR": np.exp(params),
    "OR_lower": np.exp(conf["lower"]),
    "OR_upper": np.exp(conf["upper"])
})
print("\n=== Odds Ratios ===")
print(OR_table)

# 预测 EXP 概率
data["pred_prob"] = logit_res.predict(X_logit)

# =========================
# 4. PCA（用全部自变量）
# =========================

pca = PCA(n_components=3)
PC_scores = pca.fit_transform(X_scaled)

PC_df = pd.DataFrame(
    PC_scores,
    columns=["PC1", "PC2", "PC3"],
    index=X_scaled.index
)

print("\n=== PCA Explained Variance Ratio ===")
print(pca.explained_variance_ratio_)
print("累计解释方差:", pca.explained_variance_ratio_.cumsum())

loadings = pd.DataFrame(
    pca.components_.T,
    index=X_cols,
    columns=["PC1", "PC2", "PC3"]
)
print("\n=== PCA Loadings ===")
print(loadings)

# =========================
# 5. KMeans 聚类（基于 PC1–PC3）
# =========================

k = 4
kmeans = KMeans(n_clusters=k, random_state=42)
PC_df["cluster_k4"] = kmeans.fit_predict(PC_df[["PC1","PC2","PC3"]])

print("\n=== Cluster sizes ===")
print(PC_df["cluster_k4"].value_counts())

ct = pd.crosstab(PC_df["cluster_k4"], y)
print("\n=== Cluster vs EXP_flag ===")
print(ct)

# =========================
# 6. 写回原表 & 导出
# =========================

df.loc[data.index, "pred_prob"] = data["pred_prob"]
df.loc[PC_df.index, "PC1"] = PC_df["PC1"]
df.loc[PC_df.index, "PC2"] = PC_df["PC2"]
df.loc[PC_df.index, "PC3"] = PC_df["PC3"]
df.loc[PC_df.index, "Cluster_k4"] = PC_df["cluster_k4"]

df.to_csv("ng_with_EXP_PCA_KMeans_Logit_clean.csv", index=False)

OR_table.to_csv("Logit_OR_clean.csv", index=False)
loadings.to_csv("PCA_loadings_clean.csv")

print("\n全部完成，结果已导出。")



有效样本数: 1694
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.006596148141910287
            Iterations: 122
            Function evaluations: 125
            Gradient evaluations: 122
                           Logit Regression Results                           
Dep. Variable:               Exp_flag   No. Observations:                 1694
Model:                          Logit   Df Residuals:                     1683
Method:                           MLE   Df Model:                           10
Date:                Mon, 01 Dec 2025   Pseudo R-squ.:                  0.4922
Time:                        03:55:18   Log-Likelihood:                -11.174
converged:                       True   LL-Null:                       -22.006
Covariance Type:            nonrobust   LLR p-value:                   0.01691
                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------

C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=7.
  warnings.warn(


In [7]:
OR = np.exp(logit_res.params)
OR


const              1.023627e-22
BtE400             1.626408e+06
Cent_norm          2.949833e-22
400tram_ratio      4.164132e-49
400bus_ratio       3.470902e+05
metroPointCount    1.823721e-05
busPoint_Count     1.684668e-01
access_gap         1.138552e+07
IMD_rank           6.636940e+01
olderratio         1.091423e+01
popdense           2.201435e+00
dtype: float64